In [6]:
import sys
sys.path.append("../")

from utility.potential import *
from utility.propagation import *
from tqdm import tqdm # type: ignore

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Animations

In [7]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = Propagation("standard_0_0", 0, 0, 150)
propagation.config.animation = AnimationConfig.All
propagation = propagation.into_split()

propagation.propagate()
print(propagation.get_losses())
propagation.save_savers()

j init: 0, omega init: 0, j_tot: 150
[0.13303160751633436, 0.05918912498304696]


In [6]:
print("j init: 1, omega init: 0, j_tot: 150")
propagation = Propagation("standard_1_0", 1, 0, 150)
propagation.config.animation = AnimationConfig.All
propagation = propagation.into_split()

propagation.propagate()
print(propagation.get_losses())
propagation.save_savers()

j init: 1, omega init: 0, j_tot: 150
[0.1829969537575593, 0.0562377597748811]


In [7]:
print("j init: 1, omega init: 1, j_tot: 150")
propagation = Propagation("standard_1_1", 1, 1, 150)
propagation.config.animation = AnimationConfig.All
propagation = propagation.into_split()

propagation.propagate()
print(propagation.get_losses())
propagation.save_savers()

j init: 1, omega init: 1, j_tot: 150
[0.10805345243546916, 0.060667835221461686]


In [15]:
j = 0

for omega in tqdm(range(j + 1)):
    propagation = Propagation(f"standard_{j}_{omega}", j, omega, 10)
    propagation.config.animation = AnimationConfig.Polar
    propagation.provider.transform_gamma = 0.
    propagation = propagation.into_split()

    propagation.propagate()
    propagation.save_savers()


100%|██████████| 1/1 [00:01<00:00,  1.66s/it]


In [8]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = Propagation("small_rot_scaling", 0, 0, 150)
propagation.config.animation = AnimationConfig.All
propagation.params.rot_const = 9.243165268327e-9
propagation = propagation.into_split()

propagation.propagate()
print(propagation.get_losses())
propagation.save_savers()

j init: 0, omega init: 0, j_tot: 150
[0.1447736709386087, 0.06355082082058716]


In [9]:
print("j init: 0, omega init: 0, j_tot: 150")
propagation = Propagation("big_rot_scaling", 0, 0, 150)
propagation.config.animation = AnimationConfig.All
propagation.params.rot_const = 9.243165268327e-5
propagation = propagation.into_split()

propagation.propagate()
print(propagation.get_losses())
propagation.save_savers()

j init: 0, omega init: 0, j_tot: 150
[0.11065577858275077, 0.04660656797104301]


# Single cross sections

In [ ]:
j_omega_pairs = [(0, 0), (1, 0), (1, 1)]

energy_kelvin = 3700
for j_init, omega_init in j_omega_pairs:
    losses = CumulativeLosses(j_init, energy_kelvin)
    
    for j_tot in tqdm(losses.j_totals):
        propagation = Propagation("standard", j_init, omega_init, j_tot)
        propagation.params.energy_kelvin = energy_kelvin
        propagation = propagation.into_split()

        propagation.propagate()
        losses.extract_loss(propagation)
        
    losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

100%|██████████| 50/50 [01:18<00:00,  1.57s/it]


In [19]:
j_init = 2
energy_kelvin = 3700

for omega_init in range(j_init + 1):
    losses = CumulativeLosses(j_init, energy_kelvin)
    
    for j_tot in tqdm(losses.j_totals):
        propagation = Propagation("standard", j_init, omega_init, j_tot)
        propagation.params.energy_kelvin = energy_kelvin
        propagation = propagation.into_split()

        propagation.propagate()
        losses.extract_loss(propagation)
        
    losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")


100%|██████████| 50/50 [01:17<00:00,  1.56s/it]


# Cross sections - energy dependence 

In [7]:
j_omega_pairs = [(0, 0), (1, 0), (1, 1)]
energies = [500, 1000, 2000, 3000, 4000, 5000]

In [ ]:
for j_init, omega_init in j_omega_pairs:
    for energy_kelvin in energies:
        print("energy", energy_kelvin)
        losses = CumulativeLosses(j_init, energy_kelvin)

        for j_tot in tqdm(losses.j_totals):
            propagation = Propagation("standard", j_init, omega_init, j_tot)
            propagation.config.time_step *= np.sqrt(propagation.params.energy_kelvin / energy_kelvin)
            propagation.params.energy_kelvin = energy_kelvin
            propagation = propagation.into_split()

            propagation.propagate()
            losses.extract_loss(propagation)
        losses.save_losses(f"losses_{energy_kelvin}_{j_init}_{omega_init}")

energy 500


 12%|█▏        | 6/50 [00:09<01:06,  1.51s/it]

# Convergences 

In [ ]:
j_omega_pairs = [(0, 0), (1, 0), (1, 1)]

## Time step

In [ ]:
time_steps = [5., 10., 20., 50., 100., 200., 400.]

for j_init, omega_init in j_omega_pairs:
    for time_step in time_steps:
        print("time_step", time_step)
        losses = CumulativeLosses(j_init)
        
        for j_tot in tqdm(losses.j_totals):
            propagation = Propagation("standard", j_init, omega_init, j_tot)
            propagation.config.steps_no = int(propagation.config.steps_no * propagation.config.time_step / time_step)
            propagation.config.time_step = time_step
            propagation = propagation.into_split()

            propagation.propagate()
            losses.extract_loss(propagation)
        losses.save_losses(f"time_step_{time_step}_{j_init}_{omega_init}")

time_step 100.0


100%|██████████| 50/50 [00:36<00:00,  1.36it/s]


time_step 200.0


100%|██████████| 50/50 [00:20<00:00,  2.50it/s]


time_step 400.0


...

100%|██████████| 50/50 [00:37<00:00,  1.34it/s]


time_step 200.0


100%|██████████| 50/50 [00:20<00:00,  2.47it/s]


time_step 400.0


100%|██████████| 50/50 [00:11<00:00,  4.24it/s]


## radial grid

In [ ]:
r_sizes = [128, 256, 512, 1024, 2048, 4096]

for j_init, omega_init in j_omega_pairs:
    for r_no in r_sizes:
        print("radial_no", r_no)
        losses = CumulativeLosses(j_init)

        for j_tot in tqdm(losses.j_totals):
            propagation = Propagation("standard", j_init, omega_init, j_tot)
            propagation.config.r_no = r_no
            propagation = propagation.into_split()

            propagation.propagate()
            losses.extract_loss(propagation)
        losses.save_losses(f"radial_scaling_{r_no}_{j_init}_{omega_init}")

radial_no 128


100%|██████████| 50/50 [00:29<00:00,  1.70it/s]


radial_no 256


100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


radial_no 512


...

100%|██████████| 50/50 [02:02<00:00,  2.44s/it]


radial_no 2048


100%|██████████| 50/50 [04:04<00:00,  4.90s/it]


radial_no 4096


100%|██████████| 50/50 [07:23<00:00,  8.86s/it]


## polar grid

In [ ]:
polar_sizes = [32, 64, 128, 256, 512]

for j_init, omega_init in j_omega_pairs:
    for polar_no in polar_sizes:
        print("polar_no", polar_no)
        losses = CumulativeLosses(j_init)

        for j_tot in tqdm(losses.j_totals):
            propagation = Propagation("standard", j_init, omega_init, j_tot)
            propagation.config.polar_no = polar_no
            propagation = propagation.into_split()

            propagation.propagate()
            losses.extract_loss(propagation)
        losses.save_losses(f"polar_scaling_{polar_no}_{j_init}_{omega_init}")

polar_no 32


100%|██████████| 50/50 [00:22<00:00,  2.26it/s]


polar_no 64


100%|██████████| 50/50 [00:37<00:00,  1.32it/s]


polar_no 128


...

100%|██████████| 50/50 [01:11<00:00,  1.42s/it]


polar_no 256


100%|██████████| 50/50 [02:50<00:00,  3.42s/it]


polar_no 512


100%|██████████| 50/50 [08:13<00:00,  9.87s/it]


## wave width

In [ ]:
widths = [0.1, 0.3, 0.6, 1, 2, 4]

for j_init, omega_init in j_omega_pairs:
    for width in widths:
        print("wave width", width)
        losses = CumulativeLosses(j_init)

        for j_tot in tqdm(losses.j_totals):
            propagation = Propagation("standard", j_init, omega_init, j_tot)
            propagation.params.wave_r_sigma = width
            propagation = propagation.into_split()

            propagation.propagate()
            losses.extract_loss(propagation)
        losses.save_losses(f"width_scaling_{width}_{j_init}_{omega_init}")

wave width 0.1


100%|██████████| 50/50 [01:12<00:00,  1.45s/it]


wave width 0.3


100%|██████████| 50/50 [01:10<00:00,  1.41s/it]


wave width 0.6


...

100%|██████████| 50/50 [01:10<00:00,  1.42s/it]


wave width 2


100%|██████████| 50/50 [01:10<00:00,  1.42s/it]


wave width 4


100%|██████████| 50/50 [01:10<00:00,  1.42s/it]


## Coriolis time step

In [ ]:
time_steps = [100., 200., 400., 800., 1600.]

for j_init, omega_init in j_omega_pairs:
    for time_step in time_steps:
        print("time_step", time_step)
        losses = CumulativeLosses(j_init)
        
        for j_tot in tqdm(losses.j_totals):
            propagation = Propagation("standard", j_init, omega_init, j_tot)
            propagation.config.coriolis_omega_max = 2
            propagation.config.steps_no = int(propagation.config.steps_no * propagation.config.time_step / time_step)
            propagation.config.time_step = time_step
            propagation = propagation.into_split()

            propagation.propagate()
            losses.extract_loss(propagation)
        losses.save_losses(f"coriolis_time_step_{time_step}_{j_init}_{omega_init}")

time_step 1600.0


100%|██████████| 50/50 [01:01<00:00,  1.23s/it]


time_step 1600.0


100%|██████████| 50/50 [01:00<00:00,  1.22s/it]


time_step 1600.0


100%|██████████| 50/50 [01:00<00:00,  1.21s/it]
